<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>

### 중요도에 따른 feature 정리
> 분류 확률을 계산하는데 기여한 정도를 **피처 중요도** 라고 함 <br>
> 결과에 유의미한 영향을 주는 feature 만을 중심으로 머신러닝 기법을 적용하기도 함

### 데이터 준비

In [6]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import pandas as pd

with open('titanic_step3_feature_encoding.pickle', 'rb') as pickle_filename:
    # 202402 업데이트: pandas 2.0.3 버전 이상에서는 pickle.load 실행시 에러가 남
    # 대신에 pd.read_pickle() 호출하면 됨    
    # df_onehot = pickle.load(pickle_filename)
    df_onehot = pd.read_pickle(pickle_filename)
with open('titanic_step3_feature_encoding_y.pickle', 'rb') as pickle_filename:
    # 202402 업데이트: pandas 2.0.3 버전 이상에서는 pickle.load 실행시 에러가 남
    # 대신에 pd.read_pickle() 호출하면 됨    
    # y_train = pickle.load(pickle_filename)
    y_train = pd.read_pickle(pickle_filename)
    
ntrain = 891
X_train, X_test = df_onehot[:ntrain], df_onehot[ntrain:]
X_train.head()

,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Age_0,Age_1,Age_2,Age_3,Age_4,...,HighChance_0,HighChance_1,HighChance_2,HighChance_3,HighChance_4,HighChance_5,HighChance_6,LowChance_0,LowChance_1,LowChance_2
0,0,0,1,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
1,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
2,0,0,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
3,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
4,0,0,1,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1


### XGBoost 및 LightGBM 설치

In [7]:
!pip install lightgbm

In [8]:
!pip install xgboost

### 라이브러리 모델 임포트

In [9]:
import numpy as np # 각 모델에서 내부적으로 관련 라이브러리 사용 가능
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# knn_model 은 참고차만 사용한 것이므로(본 데이터에 좋은 성능을 확인할 수 없음), 삭제하기로 함
# from sklearn.neighbors import KNeighborsClassifier             # 1. K-Nearest Neighbor(KNN)

from sklearn.linear_model import LogisticRegression            # 2. Logistic Regression
from sklearn.svm import SVC                                                # 3. SVC
from sklearn.tree import DecisionTreeClassifier                   # 4. Decision Tree
from sklearn.ensemble import RandomForestClassifier       # 5. Random Forest
from sklearn.ensemble import ExtraTreesClassifier             # 6. Extra Tree
from sklearn.ensemble import GradientBoostingClassifier  # 7. GBM
from sklearn.naive_bayes import GaussianNB                     # 8. GaussianNB
from xgboost import XGBClassifier                                     # 9. XGBoost
from lightgbm import LGBMClassifier                                 # 10. LightGBM

### 디폴트 테스트
> 하이퍼 파라미터 튜닝을 통해 각 머신러닝 모델을 보다 최적화할 수 있지만, 우선 디폴트값으로 바로 예측 가능

In [10]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# knn_model 은 참고차만 사용한 것이므로(본 데이터에 좋은 성능을 확인할 수 없음), 삭제하기로 함
# knn_model = KNeighborsClassifier()
logreg_model = LogisticRegression()
svc_model = SVC()
decision_model = DecisionTreeClassifier()
random_model = RandomForestClassifier()
extra_model = ExtraTreesClassifier()
gbm_model = GradientBoostingClassifier()
nb_model = GaussianNB()
xgb_model = XGBClassifier(eval_metric='logloss')
lgbm_model = LGBMClassifier()

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)           # K-Fold 사용
results = dict()

In [11]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# knn_model 은 참고차만 사용한 것이므로(본 데이터에 좋은 성능을 확인할 수 없음), 삭제하기로 함
# logreg_model
alg = logreg_model
alg.fit(X_train, y_train)    
score = cross_val_score(alg, X_train, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [12]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# svc_model
alg = svc_model
alg.fit(X_train, y_train)    
score = cross_val_score(alg, X_train, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [13]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# decision_model
alg = decision_model
alg.fit(X_train, y_train)    
score = cross_val_score(alg, X_train, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [14]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# random_model
alg = random_model
alg.fit(X_train, y_train)    
score = cross_val_score(alg, X_train, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [15]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# extra_model
alg = extra_model
alg.fit(X_train, y_train)    
score = cross_val_score(alg, X_train, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [16]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# gbm_model
alg = gbm_model
alg.fit(X_train, y_train)    
score = cross_val_score(alg, X_train, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [17]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# nb_model
alg = nb_model
alg.fit(X_train, y_train)    
score = cross_val_score(alg, X_train, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [18]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# xgb_model
alg = xgb_model
alg.fit(X_train, y_train)    
score = cross_val_score(alg, X_train, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

### 정확도 높은 순으로 정렬하기

- 사전 데이터의 value 로 정렬하는 방법 

In [19]:
sorted(results.items(), key=lambda x: x[1], reverse=True) # reverse=True 면 높은 순서대로 정렬

[('LogisticRegression', 83.38951310861424),
 ('SVC', 83.05118601747814),
 ('GradientBoostingClassifier', 82.71410736579276),
 ('RandomForestClassifier', 82.26466916354556),
 ('ExtraTreesClassifier', 82.26466916354556),
 ('XGBClassifier', 82.04119850187267),
 ('DecisionTreeClassifier', 80.6916354556804),
 ('GaussianNB', 70.8039950062422)]

### 성능이 좋은 머신러닝 기법만을 중심으로, 중요도 계산하기

In [20]:
tree_models = [
    random_model,
    extra_model,
    gbm_model,
    xgb_model
]

### 트리 관련 모델은 중요도가 측정됨
- 트리를 결정하는 과정에서 각 feature 가 얼마나 중요한지를 수치화하며, feature_importances_ 에 해당 값을 가지고 있음
- 해당 값을 기준으로 중요도가 낮은 feature 를 걸러낼 수 있음

In [21]:
for alg in tree_models:
    try:
        print(alg.__class__.__name__)
        print(alg.feature_importances_)        
    except:
        print(alg.__class__.__name__, "X")

RandomForestClassifier
[1.38052756e-02 1.75020749e-02 3.28671419e-02 7.28328325e-02
 6.63022983e-02 3.95863013e-03 9.13479131e-03 1.29994463e-02
 2.30645742e-02 1.86138616e-02 1.58051712e-02 5.36595920e-03
 1.17196393e-03 2.04714670e-02 1.64604037e-02 9.43206451e-03
 5.18616438e-03 3.17856734e-03 7.21258695e-03 3.27461712e-02
 7.53706852e-03 1.04306445e-02 1.56003720e-03 6.46852394e-03
 2.81417968e-03 6.22983644e-03 1.36046552e-03 3.10670250e-04
 1.87726177e-02 1.27563230e-02 8.93987683e-03 8.14623865e-02
 2.67144772e-02 2.00884741e-02 1.06610994e-02 4.08068655e-04
 2.72227481e-04 2.02975831e-03 1.34271135e-03 1.19926760e-03
 2.55894627e-04 6.37270730e-04 2.68901526e-04 1.36806045e-04
 0.00000000e+00 1.51214737e-02 1.42421802e-02 1.67241940e-02
 7.87801428e-03 6.78390984e-03 7.98172378e-03 2.53460698e-03
 1.94864634e-03 2.44270882e-03 3.71087588e-03 4.99596236e-03
 6.47117448e-03 1.29193740e-02 1.05725026e-03 3.41962611e-03
 1.15063990e-03 4.04758921e-04 4.60583084e-03 2.23425538e-03
 

- 중요도 기반 데이터프레임 작성하기

In [22]:
random_model_importance = pd.DataFrame({'Feature':X_train.columns, 'random_model':random_model.feature_importances_})
extra_model_importance = pd.DataFrame({'Feature':X_train.columns, 'extra_model':extra_model.feature_importances_})
gbm_model_importance = pd.DataFrame({'Feature':X_train.columns, 'gbm_model':gbm_model.feature_importances_})
xgb_model_importance = pd.DataFrame({'Feature':X_train.columns, 'xgb_model':xgb_model.feature_importances_})

### multiple dataframe 합치기
- dataframes = [각 데이터프레임, ...]
- functools.reduce(lambda  left,right: pd.merge(left, right, on=['동일컬럼']), dataframes)

In [23]:
from functools import reduce
data_frames = [
    random_model_importance,
    extra_model_importance,
    gbm_model_importance,
    xgb_model_importance
]
importances = reduce(lambda  left,right: pd.merge(left, right, on=['Feature']), data_frames)

In [24]:
importances.head()

,Feature,random_model,extra_model,gbm_model,xgb_model
0,Pclass_0,0.013805,0.014771,0.015736,0.007876
1,Pclass_1,0.017502,0.016864,0.000263,0.006259
2,Pclass_2,0.032867,0.035234,0.113769,0.069591
3,Sex_0,0.072833,0.081625,0.046515,0.014610
4,Sex_1,0.066302,0.057223,0.066862,0.000000


- 항목별 평균 중요도 구하기

In [25]:
# 202402 업데이트: importances.mean(axis=1) 명령 시, 데이터프레임에 숫자컬럼이 아닌 문자컬럼이 있으면 에러가 나므로
# 강제로 숫자컬럼만 추출하여, 해당 명령을 실행하기로 함

### 기존 코드
# importances['avg'] = importances.mean(axis=1)

### 신규 코드
# 숫자형 컬럼만 선택하여 평균 계산
numeric_columns = importances.select_dtypes(include=[np.number])
importances['avg'] = numeric_columns.mean(axis=1)

# 최종 데이터프레임 구성: 'Feature' 컬럼과 계산된 'avg' 컬럼 포함
importances = importances[['Feature', 'avg']]

- 중요도 기반 정렬하기

In [26]:
importances = importances.sort_values(by='avg', ascending=False)

### 중요도가 높은 feature 만 선택하기

In [27]:
importances = importances[:50]

- 특정 컬럼만 선택해서, 데이터프레임 만들기

In [28]:
train_importance = X_train[importances['Feature'].tolist()]
test_importance = X_test[importances['Feature'].tolist()]

In [29]:
train_importance.head()

,Initial_0,Pclass_2,Sex_0,Sex_1,LowChance_0,HighChance_0,Cabin_0,Fare_0,Ticket_Num_Cut_2,Family_2,...,HighChance_3,Age_1,Ticket_Num_Cut_3,Age_6,Ticket_Num_Cut_6,Cabin_4,Family_3,Ticket_Initial2_2,Fare_5,Ticket_Initial2_15
0,1,1,1,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,1,0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 중요도가 높은 feature 로만 머신러닝 적용해보기

In [30]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# knn_model 은 참고차만 사용한 것이므로(본 데이터에 좋은 성능을 확인할 수 없음), 삭제하기로 함
# knn_model = KNeighborsClassifier()
logreg_model = LogisticRegression()
svc_model = SVC()
decision_model = DecisionTreeClassifier()
random_model = RandomForestClassifier()
extra_model = ExtraTreesClassifier()
gbm_model = GradientBoostingClassifier()
nb_model = GaussianNB()
xgb_model = XGBClassifier(eval_metric='logloss')
lgbm_model = LGBMClassifier()

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)           # K-Fold 사용
results = dict()

In [31]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# knn_model 은 참고차만 사용한 것이므로(본 데이터에 좋은 성능을 확인할 수 없음), 삭제하기로 함
# logreg_model
alg = logreg_model
alg.fit(train_importance, y_train)    
score = cross_val_score(alg, train_importance, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [32]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# svc_model
alg = svc_model
alg.fit(train_importance, y_train)    
score = cross_val_score(alg, train_importance, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [33]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# decision_model
alg = decision_model
alg.fit(train_importance, y_train)    
score = cross_val_score(alg, train_importance, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [34]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# random_model
alg = random_model
alg.fit(train_importance, y_train)    
score = cross_val_score(alg, train_importance, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [35]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# extra_model
alg = extra_model
alg.fit(train_importance, y_train)    
score = cross_val_score(alg, train_importance, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [36]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# gbm_model
alg = gbm_model
alg.fit(train_importance, y_train)    
score = cross_val_score(alg, train_importance, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [37]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# nb_model
alg = nb_model
alg.fit(train_importance, y_train)    
score = cross_val_score(alg, train_importance, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [38]:
# 202402 업데이트: 여러 모델을 한번에 실행시, 에러가 날 수 있으므로, 각 모델을 별도로 실행하기로 함
# xgb_model
alg = xgb_model
alg.fit(train_importance, y_train)    
score = cross_val_score(alg, train_importance, y_train.values.ravel(), cv=k_fold, scoring='accuracy')
results[alg.__class__.__name__] = np.mean(score)*100

In [39]:
sorted(results.items(), key=lambda x: x[1], reverse=True) # reverse=True 면 높은 순서대로 정렬

[('LogisticRegression', 83.72659176029963),
 ('GradientBoostingClassifier', 83.16354556803995),
 ('SVC', 82.71535580524345),
 ('ExtraTreesClassifier', 82.14981273408239),
 ('XGBClassifier', 82.04119850187267),
 ('RandomForestClassifier', 81.81523096129837),
 ('DecisionTreeClassifier', 80.57927590511859),
 ('GaussianNB', 79.67915106117353)]

<div class="alert alert-block" style="border: 2px solid #E65100;background-color:#FFF3E0;padding:10px">
<font size="4em" style="font-weight:bold;color:#BF360C;">큰그림으로 이해하기</font><br>
<font size="4em" style="color:#BF360C;">환경에 따라 결과가 조금씩 차이가 있을 수 있지만, 제 PC에서는 최대 83.38% 예측 정확도를 보임</font><br>
<font size="4em" style="color:#BF360C;">오히려 살짝 떨어져보이지만, 하이퍼파라미터 튜닝을 통해, 성능 개선 가능</font><br>
<font size="4em" style="color:#BF360C;">반복적인 성능 개선을 위해, 불필요한 컬럼을 삭제하여, 수행속도를 높이고, 중요도 높은 컬럼만으로 결국 정확도를 높일 수 있음</font>
</div>

In [49]:
import pickle
with open('titanic_step4_importance_train.pickle', 'wb') as pickle_filename:
    pickle.dump(train_importance, pickle_filename)
with open('titanic_step4_importance_test.pickle', 'wb') as pickle_filename:
    pickle.dump(test_importance, pickle_filename)
with open('titanic_step4_importance_train_y.pickle', 'wb') as pickle_filename:
    pickle.dump(y_train, pickle_filename)

<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>